In [ ]:
pip install requests


With Weather API Key: Dynamic Weather Updates from Open Weather API

In [21]:
from kafka import KafkaProducer
import json
import time
import requests
from datetime import datetime

# Function to create a Kafka Producer
def create_producer():
    return KafkaProducer(
        bootstrap_servers='localhost:9092',
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

# Function to fetch weather data from OpenWeatherMap
def fetch_weather_data(api_key, city="New York"):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=imperial"
    response = requests.get(url)
    data = response.json()
    
    # Check for the response status
    if response.status_code != 200:
        print(f"Error fetching weather data: {data.get('message', 'No error message')}")
        return None
    
    if 'main' not in data:
        print("Missing 'main' in response. Here's the data received:", data)
        return None
    
    weather_data = {
        "city": city,
        "temperature": data["main"]["temp"],
        "description": data["weather"][0]["description"],
        "time": datetime.now().isoformat()
    }
    return weather_data

# Function to send data
def send_weather_data(producer, topic_name='weather', api_key=''):
    weather_data = fetch_weather_data(api_key)
    if weather_data:
        producer.send(topic_name, value=weather_data)
        producer.flush()
        print(f"Sent data: {weather_data}")
    else:
        print("Failed to fetch weather data")

if __name__ == '__main__':
    api_key = '4788fd85d2b57dbdb611d45c14d9e16c'  # API key
    producer = create_producer()
    while True:
        send_weather_data(producer, api_key=api_key)
        time.sleep(120)  # Send data every 2 minutes


Sent data: {'city': 'New York', 'temperature': 66.25, 'description': 'overcast clouds', 'time': '2024-04-24T14:24:05.378201'}
Sent data: {'city': 'New York', 'temperature': 66.25, 'description': 'overcast clouds', 'time': '2024-04-24T14:26:05.503063'}


KeyboardInterrupt: 

Without Using Api, sending Randomly Generated Weather data 

In [22]:
from kafka import KafkaProducer
import json
import time
import random
from datetime import datetime

# Function to create a Kafka Producer
def create_producer():
    return KafkaProducer(
        bootstrap_servers='localhost:9092',
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

# Simulate dynamic weather data
def simulate_weather_data(city):
    weather_descriptions = ["sunny", "cloudy", "rainy", "foggy", "snowy", "windy"]
    temperatures = {
        "sunny": random.uniform(75, 95),
        "cloudy": random.uniform(60, 75),
        "rainy": random.uniform(55, 65),
        "foggy": random.uniform(50, 60),
        "snowy": random.uniform(30, 40),
        "windy": random.uniform(60, 70)
    }
    
    # Randomly choose a weather condition
    description = random.choice(weather_descriptions)
    temperature = temperatures[description]

    weather_data = {
        "city": city,
        "temperature": round(temperature, 2),
        "description": description,
        "time": datetime.now().isoformat()
    }
    return weather_data

# Function to send data
def send_weather_data(producer, topic_name='weather', city='New York'):
    weather_data = simulate_weather_data(city)
    producer.send(topic_name, value=weather_data)
    producer.flush()
    print(f"Sent data: {weather_data}")

if __name__ == '__main__':
    producer = create_producer()
    while True:
        send_weather_data(producer)
        time.sleep(5)  # Send data every 5 seconds


Sent data: {'city': 'New York', 'temperature': 34.47, 'description': 'snowy', 'time': '2024-04-24T14:29:13.438978'}
Sent data: {'city': 'New York', 'temperature': 73.96, 'description': 'cloudy', 'time': '2024-04-24T14:29:18.449269'}
Sent data: {'city': 'New York', 'temperature': 57.47, 'description': 'foggy', 'time': '2024-04-24T14:29:23.461765'}
Sent data: {'city': 'New York', 'temperature': 60.48, 'description': 'rainy', 'time': '2024-04-24T14:29:28.480423'}
Sent data: {'city': 'New York', 'temperature': 63.36, 'description': 'cloudy', 'time': '2024-04-24T14:29:33.493026'}
Sent data: {'city': 'New York', 'temperature': 65.15, 'description': 'cloudy', 'time': '2024-04-24T14:29:38.503044'}
Sent data: {'city': 'New York', 'temperature': 30.1, 'description': 'snowy', 'time': '2024-04-24T14:29:43.545294'}


KeyboardInterrupt: 